In [1]:
import time
import requests
from requests.exceptions import InvalidSchema

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [28]:
def scroll_down(driver):
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        time.sleep(2)
        new_height = driver.execute_script('return document.body.scrollHeight')
        try:
            driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()
            time.sleep(2)
        except:
            pass
        if new_height == last_height:
            break
        last_height = new_height

def search_google(search_query, images_url):
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=options)
    search_url = f"https://www.google.com/search?q={search_query}&tbm=isch&tbs=isz:l"

    # open browser and accept terms and conditions
    driver.get(search_url)
    button = driver.find_element_by_xpath('//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button')
    button.click()
    
    # scroll all the way down to load all images
    scroll_down(driver)
    elements = driver.find_elements_by_class_name('rg_i')
    
    count = 0
    error_count = 0
    for e in elements:
        # get images source url
        e.click()
        time.sleep(1)
        element = driver.find_elements_by_class_name('v4dQwb')

        # Google image web site logic
        big_img = element[int(count!=0)].find_element_by_class_name('n3VNCb')
        
        # write image to file
        try:
            image_url = big_img.get_attribute("src")
            if image_url not in images_url:
                reponse = requests.get(image_url)
                images_url.append(image_url)
                with open(f"data_000/search_p3_{len(images_url)}.jpg","wb") as file:
                    file.write(reponse.content)
                print(f'{len(images_url)} files(s) saved')
                count += 1
        except:
            error_count += 1
        
        if count == 100 or error_count == 10:
            break
            
    driver.close()
    return images_url

In [1]:
items = []
for query in ['new car', 'used car', 'used vehicle', 'nowy samochód', 'używany samochód', 'nowe auto']:
    items = search_google(query, items)